#### 1. Importing Required libraries

In [20]:
from dotenv import load_dotenv

load_dotenv()
import json
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatVertexAI
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import DeepLake

#### 2. Loading the Hyd data

In [2]:
loader = TextLoader('data/articles_content.txt')

In [3]:
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=20
)

texts = text_splitter.split_documents(documents)

In [5]:
embeddings = VertexAIEmbeddings()

dataset_path = "./data/hyd_articles_embeddings"

db = DeepLake(
    embedding=embeddings, 
    dataset_path=dataset_path,
    overwrite=True
)

db.add_documents(texts)

Creating embedding data: 100%|██████████| 8/8 [00:04<00:00,  1.91it/s]


Dataset(path='./data/hyd_articles_embeddings', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (79, 768)  float32   None   
    id        text      (79, 1)     str     None   
 metadata     json      (79, 1)     str     None   
   text       text      (79, 1)     str     None   


['decbbd88-7475-11ee-b4fe-04421acf6dbe',
 'decbbd89-7475-11ee-b786-04421acf6dbe',
 'decbbd8a-7475-11ee-9552-04421acf6dbe',
 'decbbd8b-7475-11ee-8f9c-04421acf6dbe',
 'decbbd8c-7475-11ee-aebc-04421acf6dbe',
 'decbbd8d-7475-11ee-94d0-04421acf6dbe',
 'decbbd8e-7475-11ee-ad08-04421acf6dbe',
 'decbbd8f-7475-11ee-b7b5-04421acf6dbe',
 'decbbd90-7475-11ee-8407-04421acf6dbe',
 'decbbd91-7475-11ee-8c3a-04421acf6dbe',
 'decbbd92-7475-11ee-bab0-04421acf6dbe',
 'decbbd93-7475-11ee-a452-04421acf6dbe',
 'decbbd94-7475-11ee-8e30-04421acf6dbe',
 'decbbd95-7475-11ee-8f45-04421acf6dbe',
 'decbbd96-7475-11ee-b36a-04421acf6dbe',
 'decbbd97-7475-11ee-bdec-04421acf6dbe',
 'decbbd98-7475-11ee-891e-04421acf6dbe',
 'decbbd99-7475-11ee-b367-04421acf6dbe',
 'decbe4ac-7475-11ee-a2c0-04421acf6dbe',
 'decbe4ad-7475-11ee-9cab-04421acf6dbe',
 'decbe4ae-7475-11ee-85d6-04421acf6dbe',
 'decbe4af-7475-11ee-8526-04421acf6dbe',
 'decbe4b0-7475-11ee-bf90-04421acf6dbe',
 'decbe4b1-7475-11ee-947b-04421acf6dbe',
 'decbe4b2-7475-

In [6]:
"""db_loaded = DeepLake(
    dataset_path=dataset_path, 
    embedding_function=embeddings, 
    token=os.getenv("ACTIVELOOP_TOKEN"),
    org_id=os.getenv("ACTIVELOOP_ORG_ID"),
    read_only=True
)"""

retriever = db.as_retriever()

In [11]:
template = """
            You are an expert intelligent and creative AI itinerary planner with extensive knowledge of places worldwide. Your goal is to plan an optimized itinerary for the user based on their specific interests and preferences, geographical proximity, and efficient routes to minimize travel time. To achieve this, follow these instructions:

            1. Suggest three activities along with two restaurant per day. Each activity should include the name of the place, a brief description, estimated cost, and time to visit the place.

            2. Generate a well-structured itinerary including day-to-day activities, timings to visit each location, and estimated costs for the user's reference.

            3. Take into account factors such as geographical proximity between destinations, transportation options, and other logistical considerations when planning the route.
            
            By following these guidelines, you will create a comprehensive and optimized itinerary that meets the user's expectations while ensuring minimal travel time.
          Use this context to generate the itinerary:
          {context}
        
          Question: {query}
          Answer:"""

In [13]:
prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=template,
    )

llm = ChatVertexAI(temperature=0.7)

In [24]:
query = """Generate 3 day itinerary for a trip to Hyderabad, India. Consider budget, timings and requirements. Include estimated cost and timings to visit for each activity and restaurant.
    Structure the itinerary as follows:
    {{"Name":"name of the trip", "description":"description of the entire trip", "budget":"budget of the entire thing", "data": [{{"day":1, "day_description":"Description based on the entire day's places. in a couple of words, for example: `Urban Exploration`, `Historical Exploration`, `Spiritual Tour`, `Adventurous Journey`, `Dayout in a beach`, `Wildlife Safari`, `Artistic Getaway`, `Romantic Getaway`, `Desert Safari`, `Island Hopping Adventure`...",  "places":[{{"name":"Place Name", "description":"Place Description in two lines","time_to_visit": "time to visit this place, for example: 9:00 to 10:00", "budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "time to visit this place, for example 10:30 - 13:00", "budget":"cost"}}]}}, {{"day":2, "day_description": "Description based on the entire day's places", "places":[{{"name":"Place Name","description":"Place Description in two lines","time_to_visit": "time to visit this place","budget":"cost"}}, {{"name":"Place Name 2", "description":"Place Description 2 in two lines","time_to_visit": "time to visit this place","budget":"cost"}}]}}]}}
    Note: Do not include any extra information outside this structure."""

In [25]:
docs = db.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]

In [35]:
prompt_ = prompt.format(context=retrieved_chunks, query=query)


llm = VertexAI(model_name='text-bison', temperature=0.7, max_output_tokens=2004)

In [36]:
response = llm(prompt_)

In [37]:
print(response)

 {{
"Name": "Hyderabad Trip",
"description": "Trip to explore the rich history and culture of Hyderabad.",
"budget": "$1000",
"data": [
{
"day": 1,
"day_description": "Historical Exploration",
"places": [
{
"name": "Charminar",
"description": "Awe-inspiring architecture built in 1591, a symbol of Hyderabad.",
"time_to_visit": "10:00 to 12:00",
"budget": "$5"
},
{
"name": "Golkonda Fort",
"description": "An archeological gem, a fort built in the 16th century.",
"time_to_visit": "12:00 to 14:00",
"budget": "$10"
},
{
"name": "Mecca Masjid",
"description": "A heritage site, one of the largest mosques in India.",
"time_to_visit": "14:00 to 16:00",
"budget": "$2"
},
{
"name": "Qutb Shahi Tombs",
"description": "Marvel at the Persian architecture, built in the 16th century.",
"time_to_visit": "16:00 to 18:00",
"budget": "$5"
}
]
},
{
"day": 2,
"day_description": "Nature and Adventure",
"places": [
{
"name": "Hussain Sagar Lake",
"description": "Mesmerizing views, a beautiful lake in the hear